# Model Training - FastText

In [10]:
# Import required libraries
import numpy as np
import pandas as pd

from gensim.models import FastText
from sklearn.metrics.pairwise import cosine_similarity
import mlflow

import json
import requests
import boto3

import streamlit as st

import warnings
warnings.filterwarnings("ignore")

In [11]:
import yaml

def load_params():
    with open("../params.yaml") as f:
        params = yaml.safe_load(f)
    return params

params = load_params()

In [12]:
def connect_database():
    """Connect to the DynamoDB database."""
    dynamodb = boto3.resource("dynamodb",
                            aws_access_key_id=st.secrets.s3.AWS_ACCESS_KEY_ID,
                            aws_secret_access_key=st.secrets.s3.AWS_SECRET_ACCESS_KEY,
                            region_name=st.secrets.s3.AWS_DEFAULT_REGION)
    table = dynamodb.Table(st.secrets.s3.DB_NAME)
    return table

data = pd.read_pickle(params["model_pipeline"]["recipe_path"])
table = connect_database()

## Model Training

In [17]:
# fasttext_model = FastText(data.ingredients, **params["model_pipeline"]["fast_text"])
# fasttext_model.save(params["model_pipeline"]["model_path"])
fasttext_model = FastText.load(params["model_pipeline"]["model_path"])

In [18]:
# Quick sanity check
fasttext_model.wv.most_similar("peanut butter", topn=10)

[('peanut_butter', 0.8835346102714539),
 ('smooth_peanut_butter', 0.8488717079162598),
 ('creamy_peanut_butter', 0.780326783657074),
 ('salted_creamy_peanut_butter', 0.7235647439956665),
 ('creamy_salted_peanut_butter', 0.7070525884628296),
 ('cocoa_butter', 0.6918313503265381),
 ('butter', 0.6805139183998108),
 ('nut_butter', 0.6800909638404846),
 ('salted_natural_peanut_butter', 0.6717429161071777),
 ('natural_salted_peanut_butter', 0.6415278911590576)]

### Wrap Model with MLFlow Python Model

Models from Code (Add set model) or Load as Class (Restart kernel after every overwrite)

In [19]:
from gensim.models import FastText
import mlflow

class RecipeFastText(mlflow.pyfunc.PythonModel):
    def __init__(self, params):
        self.params = params
    
    def train_save(self, data):
        fasttext_model = FastText(data, **self.params["fast_text"])
        fasttext_model.save(self.params["model_path"])
        
    def load_context(self, context):
        self.model = FastText.load(context.artifacts["model_path"])

    def predict(self, model_input: list[str]) -> list[float]:
        """Predicts the model vector based on the input string."""
        return [self.model.wv.get_vector(text) for text in model_input]

### Log Model to MLFlow

In [20]:
mlflow.set_tracking_uri("http://127.0.0.1:8080")
mlflow.set_experiment("model_training")

<Experiment: artifact_location='mlflow-artifacts:/329658568731902423', creation_time=1755993837497, experiment_id='329658568731902423', last_update_time=1755993837497, lifecycle_stage='active', name='model_training', tags={}>

In [21]:
with mlflow.start_run():
    # Load data
    data = pd.read_pickle(params["model_pipeline"]["recipe_path"])

    # Create fasttext model
    fasttext_model = RecipeFastText(params["model_pipeline"])

    # Train model
    fasttext_model.train_save(data.ingredients)

    # Log parameters
    mlflow.log_params(params["model_pipeline"])

    # Log model
    model_info = mlflow.pyfunc.log_model(
        name="fasttext_model",
        python_model=fasttext_model,
        artifacts={"model_path": params["model_pipeline"]["model_path"],
                   "model_ngram_path": params["model_pipeline"]["model_ngram_path"]},
        pip_requirements=["gensim==4.3.3"]
    )

🏃 View run clean-shoat-126 at: http://127.0.0.1:8080/#/experiments/329658568731902423/runs/fa0086b76a724afb833fecb302bd9cb3
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/329658568731902423


In [9]:
# model = mlflow.pyfunc.load_model(model_info.model_uri)
model = mlflow.pyfunc.load_model("models:/fasttext_model/latest")

### IDF (Inverse Document Frequency)

In [5]:
idf = {}
vector_embedding = {}
for i, vocab in enumerate(fasttext_model.wv.index_to_key):
    # Apply IDF
    n_recipe_contains_vocab = data.ingredients.apply(lambda x: vocab in x).sum()
    idf[vocab] = np.log(len(data.ingredients) / n_recipe_contains_vocab)

    # Apply IDF weight to Word2Vec embeddings
    vector_embedding[vocab] = fasttext_model.wv.vectors[i]/np.linalg.norm(fasttext_model.wv.vectors[i]) * idf[vocab] # Normalize w2v embedding first
    vector_embedding[vocab] = vector_embedding[vocab] / np.linalg.norm(vector_embedding[vocab])  # Normalize combined embedding again

idf

{'salt': 0.3928653931060427,
 'garlic': 0.9728863994977894,
 'water': 1.2474342884510103,
 'olive_oil': 1.3923734071962728,
 'black_pepper': 1.400105404479599,
 'sugar': 1.7065040189406901,
 'egg': 1.7908029890020982,
 'soy': 1.8923467061332238,
 'onion': 2.013707563137491,
 'ginger': 2.071498688165899,
 'vanilla_extract': 2.1601224114682016,
 'lemon': 2.1754537222149906,
 'unsalted_butter': 2.2039456780092967,
 'carrot': 2.3909022373074,
 'allpurpose_flour': 2.4993647329632305,
 'parsley': 2.522941463527227,
 'baking': 2.5289235352047745,
 'maple_syrup': 2.5349416075303375,
 'scallion': 2.571831036216051,
 'sesame_oil': 2.571831036216051,
 'milk': 2.6079667425452273,
 'pepper': 2.6298494537947352,
 'cinnamon': 2.717665659801652,
 'white_pepper': 2.7470795450079453,
 'cilantro': 2.759595352939776,
 'vegetable_oil': 2.7646458687258444,
 'oil': 2.7876942641621354,
 'cornstarch': 2.830028627988696,
 'lime': 2.846377765990226,
 'neutral_oil': 2.8518874218011954,
 'shaoxing_wine': 2.8999872

In [6]:
# Check if it's normalized
np.linalg.norm(vector_embedding.get("salt"))

1.0

### Model Scoring System

In [23]:
model.predict(["salt"])

[array([-2.6550920e+00, -2.8948134e-01, -1.3038169e+00, -1.2672992e+00,
        -1.1699896e-01, -2.4807813e+00, -5.7276672e-01, -2.0239155e+00,
        -3.6649773e+00, -2.4176674e+00, -2.5293906e+00,  1.3403758e+00,
         7.7194236e-02,  5.2182901e-01, -9.9768186e-01, -2.1142140e-01,
         2.2082214e+00, -2.7719632e-01,  1.8602798e+00,  2.0711403e+00,
         8.0471212e-01,  1.6247444e+00, -3.2902260e+00,  9.0726924e-01,
         1.5361571e+00, -7.8738576e-01, -2.5842564e+00,  1.7833999e+00,
        -2.7365606e+00, -2.5798705e+00, -2.3482225e+00, -9.8127532e-01,
        -1.1024963e+00,  1.2330235e+00, -1.2888752e+00, -1.0355765e+00,
        -1.5144287e+00, -9.3993258e-01,  4.2902217e+00,  8.4660459e-01,
         2.4824442e-01,  5.2841846e-02, -7.7579081e-01,  1.3014902e+00,
        -2.3578355e+00,  8.9016682e-01, -1.4454038e+00, -8.0663610e-01,
         1.3427621e+00, -1.5135692e+00,  2.1927497e+00,  1.4676518e+00,
        -1.7562844e+00,  1.6300368e+00, -2.5004225e+00,  2.20193

In [24]:
# Prepare ingredient vectors
ingredient_vectors = []
for ingredients in data.ingredients:
    # Get the vectors for the recipe ingredients and normalize them
    embedding_vecs = model.predict(ingredients)
    unit_vecs = np.array([vec/np.linalg.norm(vec) for vec in embedding_vecs])
    ingredient_vectors.append(unit_vecs)

ingredient_vector_means = np.array([np.mean(ing_vector, axis=0) for ing_vector in ingredient_vectors])

In [25]:
# Prepare title vectors
title_vectors = []
for title in data.recipe_title:
    # Get the vector for the recipe title and normalize it
    title_split = title.split()
    embedding_vecs = model.predict(title_split)
    unit_vecs = np.array([vec/np.linalg.norm(vec) for vec in embedding_vecs])
    title_vectors.append(unit_vecs)

title_vector_means = np.array([np.mean(ing_vector, axis=0) for ing_vector in title_vectors])

In [26]:
def normalization(arr):
    return (arr - arr.min()) / (arr.max() - arr.min())

In [ ]:
# Prediction scoring system
user_id = 110833230122006731136
n=10
text = "chicken thighs, rice, peanut butter, almond milk, kecap manis, soy sauce"
query = [ingredient.strip() for ingredient in text.split(",") if ingredient]
print("query:", query)

# Get the vectors for the search query and normalize them
query_vecs = fasttext_model.predict(query)
unit_query_vec = np.array([vec/np.linalg.norm(vec) for vec in query_vecs])

# Calculate MaxSim for each recipe in the data
# Might be slow for large amount of data
ingredient_max_sim = np.array([])
for ing_vector in ingredient_vectors:
    tokens_sim = unit_query_vec @ ing_vector.T

    a_best = tokens_sim.max(axis=1)
    b_best = tokens_sim.max(axis=0)
    
    score = 0.5 * (a_best.mean() + b_best.mean())
    ingredient_max_sim = np.append(ingredient_max_sim, score)

# Calculate cosine similarity between the mean query vector and the mean ingredients vector
mean_query_vec = unit_query_vec.mean(axis=0)
ingredient_sim = mean_query_vec.reshape(1, -1) @ ingredient_vector_means.T
title_sim = mean_query_vec.reshape(1, -1) @ title_vector_means.T

score = (params["model_service"]["w_cosine"] * normalization(ingredient_sim[0]) + 
            params["model_service"]["w_maxsim"] * normalization(ingredient_max_sim) + 
            params["model_service"]["w_title"] * normalization(title_sim[0]))

rec_idx = np.argsort(score)[-params["model_service"]["n_recs"]:][::-1]
# rec_idx = [4557, 5297, 3977, 6109, 366, 3572, 6262, 314]
data.iloc[rec_idx]

query: ['chicken thighs', 'rice', 'peanut butter', 'almond milk', 'kecap manis', 'soy sauce']


,id,recipe_title,recipe_url,ingredients,num_steps,total_time,prep_time,cook_time,custom_time,calories,...,sodium,potassium,fiber,sugar,vitamin_a,vitamin_c,calcium,iron,serving_size,image_url
4725,4725,Velveting Chicken The Chinese Restaurant Secret,https://www.recipetineats.com/velveting-chicke...,"[chicken_breast_slice_size_piece, baking_soda]",6.0,25.0,5.0,0.0,20.0,95.0,...,411.0,308.0,0.0,0.0,25.0,1.0,4.0,0.3,0.0,https://www.recipetineats.com/tachyon/2019/03/...
5558,5558,Chicken Thighs,https://thewoksoflife.com/how-to-debone-chicke...,[chicken_thigh],4.0,10.0,10.0,0.0,0.0,211.0,...,74.0,198.0,0.0,0.0,75.0,0.0,8.0,1.0,0.0,https://thewoksoflife.com/wp-content/uploads/2...
5484,5484,Down A Chicken,https://thewoksoflife.com/how-to-break-down-a-...,[chicken],9.0,10.0,10.0,0.0,0.0,409.0,...,133.0,360.0,0.0,0.0,267.0,3.0,21.0,2.0,0.0,https://thewoksoflife.com/wp-content/uploads/2...
5297,5297,Rice Flour,https://thewoksoflife.com/how-to-make-rice-flour/,[rice],2.0,5.0,5.0,0.0,0.0,293.0,...,0.0,61.0,2.0,0.1,0.0,0.0,8.0,0.3,0.5,https://thewoksoflife.com/wp-content/uploads/2...
4557,4557,Rice S Outrageously Delicious Rice,https://www.recipetineats.com/rice-recipes-ric...,"[one_pot_greek_chicken, lemon_rice, mexican_be...",7.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.recipetineats.com/tachyon/2020/03/...
4504,4504,Chicken Broccoli Stir Fry,https://www.recipetineats.com/chicken-broccoli...,"[chicken_breast, baking_soda_bicarb, cornstarc...",7.0,16.0,10.0,6.0,0.0,239.0,...,1045.0,660.0,2.0,2.0,525.0,73.0,50.0,1.0,0.0,https://www.recipetineats.com/tachyon/2020/07/...
5488,5488,Chicken And Mushroom Clay Pot Rice,https://thewoksoflife.com/chicken-mushroom-cla...,"[shiitake_mushroom, lily_flower, wood_ear_mush...",6.0,190.0,165.0,25.0,0.0,595.0,...,927.0,579.0,3.0,2.0,87.0,1.0,48.0,3.0,0.0,https://thewoksoflife.com/wp-content/uploads/2...
4521,4521,Jasmine Rice,https://www.recipetineats.com/how-to-cook-jasm...,"[jasmine_rice, water]",7.0,23.0,1.0,12.0,10.0,169.0,...,6.0,53.0,1.0,1.0,0.0,0.0,13.0,1.0,0.0,https://www.recipetineats.com/tachyon/2020/06/...
4533,4533,Chinese Noodle Soup,https://www.recipetineats.com/chinese-noodle-s...,"[chicken_broth_low_sodium, garlic, ginger_piec...",6.0,15.0,5.0,10.0,0.0,352.0,...,1067.0,493.0,2.0,6.0,29.0,1.0,28.0,4.0,585.0,https://www.recipetineats.com/tachyon/2016/06/...
5787,5787,Shanghai Home-Style Noodle Soup,https://thewoksoflife.com/shanghai-home-style-...,"[pork_shoulder_chicken_thigh_julienned, water,...",4.0,60.0,25.0,35.0,0.0,315.0,...,366.0,621.0,2.0,4.0,704.0,15.0,28.0,2.0,0.0,https://thewoksoflife.com/wp-content/uploads/2...


### DynamoDB Data Revtrieval

In [10]:
# user_id = 110833230122006731136
# table_test = table.get_item(Key={"user_id": int(user_id)})["Item"]

# table_test

In [11]:
# from datetime import datetime
# test_map = {"liked_idx": list(map(int, list(liked_idx))), 
#             "disliked_idx": list(map(int, list(disliked_idx))),
#             "date": datetime.now().strftime("%Y-%m-%d" + "%H:%M:%S")}
# test_map


## Model Register

In [22]:
with mlflow.start_run():
    mlflow.register_model(
        model_uri=model_info._model_uri,
        name="fasttext_model"
    )

Successfully registered model 'fasttext_model'.
2025/08/23 17:04:34 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: fasttext_model, version 1


🏃 View run rumbling-snail-782 at: http://127.0.0.1:8080/#/experiments/329658568731902423/runs/769ba51e2ca54d7080dd69d61e90b4bd
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/329658568731902423


Created version '1' of model 'fasttext_model'.


In [23]:
model = mlflow.pyfunc.load_model("models:/fasttext_model/latest")